### 목적 : 대화형 챗봇 구현. 
1. 사용자의 말을 인식
2. 사용자가 미세먼지 정보를 원하는지, 잡담을 원하는지에 대한 인식
3. 사용자가 미세먼지 정보를 원할 경우
> 3-1 지역 정보를 정확히 인식, 인식 못할 경우 다시 물어보아 오타 혹은 지역명에 대한 확인 요구
4. 사용자가 잡담을 원하는 경우
> 4-1. 이후 기능 구현

#### 가설 1. word2vec의 CBOW
- 2번 단계, 사용자가 정보를 원하는지 혹은 잡담을 원하는지에 대한 구분
- text 파일 사용, 성능 향상을 위한 특수 기호 및 영어의 경우 stemming 사용
- 이후 tokenization 수행, 이 때 tokenization은 string -> list를 의미.  
ex) [['token', 'token'], ['token', 'token']]

In [5]:
import codecs
# from konlpy.tag import Twitter
# twitter = Twitter()

from gensim.models import Word2Vec
import multiprocessing

Codec이란?
- 파이썬에서 일반적으로 유니코드를 다룰 때 사용하는 decode, encode는 대용량 데이터인 경우, 모든 데이터가 메모리에 올라감
- ** codecs ** 모듈을 통해 파일을 조금씩 읽어, 메모리에도 약간의 데이터만 올려줄 수 있음

In [15]:
#서울 주소 정보 가져오기(구, 동)
test = open("datas/address_seoul.txt", 'r')
address_list = []
lines = test.readlines()
for line in lines : 
    line = (line).split("|")
    address_list.append(line[3 : 5 + 1])

In [6]:
#word2vec parameters
parameters = {
    "sg" : 0, # skip-gram
    "size" : 100, # dimesionality of the feature vectors
    "window" : 3,
    "alpha" :0.01, #learning rate
    "batch_words" : 10000, #사전 구축 시 한 번에 몇 개의 단어를 읽을 것인가
    "iter" : 10, # epoch과 유사
    "workers" : multiprocessing.cpu_count(), #cpu
    
}

In [7]:
model = Word2Vec(**parameters)

In [23]:
model.build_vocab(address_list)

In [25]:
model.train(address_list, total_examples = 100, epochs = 100, )

(84085820, 195361500)

In [58]:
model.save('model')

In [9]:
model = Word2Vec.load('model')

In [10]:
model.wv.most_similar("월계동", topn = 3)

[('중계동', 0.9998157024383545),
 ('공릉동', 0.9992175102233887),
 ('상계동', 0.9939234256744385)]

In [55]:
model.wv.similarity("흑석동", "도봉구")

0.38406586802287679

In [51]:
model.wv.most_similar(positive = ["흑석동", "종로구"], negative = ["상도동"])

[('신월동', 0.44938164949417114),
 ('목동', 0.4383370578289032),
 ('신정동', 0.42917102575302124),
 ('양천구', 0.42728814482688904),
 ('옥인동', 0.4272083640098572),
 ('도봉구', 0.4132745862007141),
 ('누상동', 0.40563732385635376),
 ('시흥동', 0.3965455889701843),
 ('독산동', 0.3907299339771271),
 ('창동', 0.3893088698387146)]

#### Comment : 
 word2vec 실습 결과 깨달은 사실
- word2vec 기본은 center word를 바탕으로 주변 context를 판단하거나(skip-gram), 주변 context를 바탕으로 center word를 판단하는(CBOW) 방식이 있음
- 구현해야 하는 기능은 **오타가 없다는 전제 하에, API가 인식하지 못하는 지역명을 바로 잡아주는 것(예를 들어, 충북, 구로5동 등)**
- 해결 아이디어 : 전국 시,도 등 주소가 적힌 csv파일을 찾아 훈련시켜볼 것

갑자기 생각난 아이디어!  
- 오타가 없다는 전제 하에 API가 인식하지 못하는 지역명 바로잡기
> 전국 주소를 word2vec으로 학습
- 이외 데이터 학습은 seq2seq로